In [6]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
# need to deal with O365 access
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('AwardsBySchool-3.csv')
#df.head()

In [7]:
# convert date column to something that pandas understands
df["Award Date"] = pd.to_datetime(df["Award Date"])
df['Year'] = pd.DatetimeIndex(df['Award Date']).year  # add year column
#df.head()

In [8]:
available_programmes = df['Programme Title'].unique()
available_years =  df['Year'].unique()
degree_types = df['Award Short Name'].unique()
degree_classes = df['Class Name Short'].unique()

In [4]:
app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='programme-name',
                options=[{'label': i, 'value': i} for i in available_programmes],
                value='Aeronatical Engineering'
            ),
            dcc.RadioItems(
                id='summary-type',
                options=[{'label': i, 'value': i} for i in ['Combined', 'MEng/BEng split']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),


    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    )
]),])

  
@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('programme-name', 'value'),
     Input('year--slider', 'value'),
     Input('summary-type','value')])
# there should be a function parameter for each of the above inputs. These are positional, so order matters rather than name.
def update_graph(programme_name,
                 year_value,
                 summary_type):
    dff = df.loc[df['Year'] == year_value] # find rows that have the right year value
    # Need to build up the numbers as lists. 
    
    if summary_type == 'Combined':
        nDegree_class=[]
        for idx, degree_class in enumerate(degree_classes):# loop through list of degree classes and count occurances of each one in dff
          nDegree_class.append(sum((dff["Programme Title"]== programme_name) & (dff["Class Name Short"] == degree_class)))
  
        fig = px.bar(x = degree_classes,
                     y = nDegree_class)
                     
        # fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

        fig.update_xaxes(title='Degree Classification')#), 
                     #type='linear') # if xaxis_type == 'Linear' else 'log') 

        fig.update_yaxes(title='Number', 
                         type='linear') # if yaxis_type == 'Linear' else 'log')
     
    else:
        nMEng_class=[]
        nBEng_class=[]
        for idx, degree_class in enumerate(degree_classes):# loop through list of degree classes and count occurances of each one in dff
            nMEng_class.append(sum((dff["Programme Title"]== programme_name) & (dff["Class Name Short"] == degree_class) & 
                                   (dff["Award Short Name"] == 'M.Eng.')))
            nBEng_class.append(sum((dff["Programme Title"]== programme_name) & (dff["Class Name Short"] == degree_class) & 
                                   (dff["Award Short Name"] == 'B.Eng.')))
  
        fig = go.Figure(data=[
           go.Bar(name='MEng', x = degree_classes, y = nMEng_class),
           go.Bar(name='BEng', x = degree_classes, y = nBEng_class)])
       
                     
   
        fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest', barmode='stack')

        fig.update_xaxes(title='Degree Classification')#), 
                     #type='linear') # if xaxis_type == 'Linear' else 'log') 

        fig.update_yaxes(title='Number', 
                         type='linear') # if yaxis_type == 'Linear' else 'log')

    return fig



if __name__ == '__main__':
    app.run_server(debug=True, port=8053)

Dash app running on http://127.0.0.1:8053/
